In [1]:
import sys
import io
import time
import requests
import datetime
import numpy as np
import pandas as pd

sys.path.append('../')
from src.util.functions import Functions

# Disable warning
pd.options.mode.chained_assignment = None 

level = 'regioni'

path_data = '../data'
path_csv = '{path_data}/csv'.format(path_data=path_data)

csv_out_name = '{path_csv}/covid19-{level}.csv'.format(path_csv=path_csv, level=level)
pop_regions_name = '{path_csv}/popolazione-regioni.csv'.format(path_csv=path_csv)
icu_regions_name = '{path_csv}/terapie-intensive.csv'.format(path_csv=path_csv)
regions_name = '{path_csv}/regioni.csv'.format(path_csv=path_csv)

In [2]:
url_cum = Functions.get_url_cum(level)
url_cum

'https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv'

In [3]:
df_raw_r = pd.read_csv(url_cum)
df_raw_r

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,0,0,0,NaN,NaN,0,5,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,0,0,0,NaN,NaN,0,0,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,0,0,0,NaN,NaN,0,1,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,0,0,0,NaN,NaN,0,10,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,18,0,0,NaN,NaN,18,148,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5665,2020-11-19T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1532,240,1772,31809,...,1479,1871,13763,1055,30124.0,18275.0,48399,856483,590160.0,NaN
5666,2020-11-19T17:00:00,ITA,9,Toscana,43.769231,11.255889,1814,287,2101,52023,...,14,1972,32480,2073,69971.0,18706.0,88677,1403341,888853.0,effettuati 54517 tamponi rapidi antigenici di...
5667,2020-11-19T17:00:00,ITA,10,Umbria,43.106758,12.388247,369,75,444,10941,...,111,556,8889,293,5727.0,14840.0,20567,377286,210121.0,NaN
5668,2020-11-19T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,144,12,156,1879,...,-58,91,3371,271,5042.0,635.0,5677,52636,30688.0,NaN


In [4]:
# Drop useless columns
cols_to_drop = ['stato', 'lat', 'long', 'note']
df_rd = df_raw_r.drop(cols_to_drop, axis=1)
df_rd

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-02-24T18:00:00,13,Abruzzo,0,0,0,0,0,0,0,0,0,NaN,NaN,0,5,NaN
1,2020-02-24T18:00:00,17,Basilicata,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,NaN
2,2020-02-24T18:00:00,18,Calabria,0,0,0,0,0,0,0,0,0,NaN,NaN,0,1,NaN
3,2020-02-24T18:00:00,15,Campania,0,0,0,0,0,0,0,0,0,NaN,NaN,0,10,NaN
4,2020-02-24T18:00:00,8,Emilia-Romagna,10,2,12,6,18,0,18,0,0,NaN,NaN,18,148,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5665,2020-11-19T17:00:00,19,Sicilia,1532,240,1772,31809,33581,1479,1871,13763,1055,30124.0,18275.0,48399,856483,590160.0
5666,2020-11-19T17:00:00,9,Toscana,1814,287,2101,52023,54124,14,1972,32480,2073,69971.0,18706.0,88677,1403341,888853.0
5667,2020-11-19T17:00:00,10,Umbria,369,75,444,10941,11385,111,556,8889,293,5727.0,14840.0,20567,377286,210121.0
5668,2020-11-19T17:00:00,2,Valle d'Aosta,144,12,156,1879,2035,-58,91,3371,271,5042.0,635.0,5677,52636,30688.0


In [5]:
last_date_str = df_rd.iloc[-1].data.split('T')[0]
last_date = datetime.datetime.strptime(last_date_str, '%Y-%m-%d')
prev_date = Functions.get_previous_date(last_date)
prev_date_str = datetime.datetime.strftime(prev_date, '%Y-%m-%d')
last_date_str, prev_date_str

('2020-11-19', '2020-11-18')

In [6]:
# Current state S
df_raw_r_1 = df_rd[df_rd.data.str.startswith(last_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_1

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-19T17:00:00,1,Piemonte,5147,390,5537,71700,77237,2173,5349,57609,5331,58272.0,81905.0,140177,1364685,865428.0
1,2020-11-19T17:00:00,2,Valle d'Aosta,144,12,156,1879,2035,-58,91,3371,271,5042.0,635.0,5677,52636,30688.0
2,2020-11-19T17:00:00,3,Lombardia,8291,915,9206,146374,155580,3497,7453,172847,20015,261506.0,86936.0,348442,3682509,2208742.0
3,2020-11-19T17:00:00,5,Veneto,2121,279,2400,64182,66582,2276,3753,43052,3057,28879.0,83812.0,112691,2624768,1011752.0
4,2020-11-19T17:00:00,6,Friuli Venezia Giulia,485,48,533,11112,11645,891,1197,9697,580,18558.0,3364.0,21922,643492,259446.0
5,2020-11-19T17:00:00,7,Liguria,1319,119,1438,15589,17027,259,792,26789,2178,31765.0,14229.0,45994,548215,273332.0
6,2020-11-19T17:00:00,8,Emilia-Romagna,2489,244,2733,58276,61009,1690,2160,31587,5218,62529.0,35285.0,97814,1935478,1003409.0
7,2020-11-19T17:00:00,9,Toscana,1814,287,2101,52023,54124,14,1972,32480,2073,69971.0,18706.0,88677,1403341,888853.0
8,2020-11-19T17:00:00,10,Umbria,369,75,444,10941,11385,111,556,8889,293,5727.0,14840.0,20567,377286,210121.0
9,2020-11-19T17:00:00,11,Marche,526,78,604,14252,14856,103,667,9095,1150,19186.0,5915.0,25101,381419,222600.0


In [7]:
# State S - 1
df_raw_r_0 = df_rd[df_rd.data.str.startswith(prev_date_str)].sort_values(by='codice_regione').reset_index(drop=True)
df_raw_r_0

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-18T17:00:00,1,Piemonte,5208,388,5596,69468,75064,1278,3281,54511,5253,56287.0,78541.0,134828,1322809,836881.0
1,2020-11-18T17:00:00,2,Valle d'Aosta,146,17,163,1930,2093,-70,87,3234,259,4966.0,620.0,5586,51936,30337.0
2,2020-11-18T17:00:00,3,Lombardia,8323,903,9226,142857,152083,-4484,7633,169056,19850,257237.0,83752.0,340989,3644914,2193319.0
3,2020-11-18T17:00:00,5,Veneto,2079,274,2353,61953,64306,1235,2972,41613,3019,28335.0,80603.0,108938,2607348,1005997.0
4,2020-11-18T17:00:00,6,Friuli Venezia Giulia,470,49,519,10235,10754,333,796,9404,567,17665.0,3060.0,20725,636812,257516.0
5,2020-11-18T17:00:00,7,Liguria,1371,118,1489,15279,16768,237,775,26271,2163,31254.0,13948.0,45202,542028,271234.0
6,2020-11-18T17:00:00,8,Emilia-Romagna,2449,244,2693,56626,59319,2051,2371,31167,5168,61355.0,34299.0,95654,1916548,995472.0
7,2020-11-18T17:00:00,9,Toscana,1805,282,2087,52023,54110,-42,2508,30573,2022,68667.0,18038.0,86705,1385511,880609.0
8,2020-11-18T17:00:00,10,Umbria,368,74,442,10832,11274,30,501,8451,286,5646.0,14365.0,20011,371777,208149.0
9,2020-11-18T17:00:00,11,Marche,527,79,606,14147,14753,242,479,8542,1139,18594.0,5840.0,24434,377624,220387.0


In [8]:
# New df regions state S with Trentino-Alto Adige
df_r_1 = Functions.replace_bt_with_taa(df_raw_r_1)
df_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,2020-11-19T17:00:00,1,Piemonte,5147,390,5537,71700,77237,2173,5349,57609,5331,58272.0,81905.0,140177,1364685,865428.0
1,2020-11-19T17:00:00,2,Valle d'Aosta,144,12,156,1879,2035,-58,91,3371,271,5042.0,635.0,5677,52636,30688.0
2,2020-11-19T17:00:00,3,Lombardia,8291,915,9206,146374,155580,3497,7453,172847,20015,261506.0,86936.0,348442,3682509,2208742.0
3,2020-11-19T17:00:00,4,Trentino-Alto Adige,876,78,954,13236,14190,429,962,17724,994,26477.0,6431.0,32908,626406,NaN
4,2020-11-19T17:00:00,5,Veneto,2121,279,2400,64182,66582,2276,3753,43052,3057,28879.0,83812.0,112691,2624768,1011752.0
5,2020-11-19T17:00:00,6,Friuli Venezia Giulia,485,48,533,11112,11645,891,1197,9697,580,18558.0,3364.0,21922,643492,259446.0
6,2020-11-19T17:00:00,7,Liguria,1319,119,1438,15589,17027,259,792,26789,2178,31765.0,14229.0,45994,548215,273332.0
7,2020-11-19T17:00:00,8,Emilia-Romagna,2489,244,2733,58276,61009,1690,2160,31587,5218,62529.0,35285.0,97814,1935478,1003409.0
8,2020-11-19T17:00:00,9,Toscana,1814,287,2101,52023,54124,14,1972,32480,2073,69971.0,18706.0,88677,1403341,888853.0
9,2020-11-19T17:00:00,10,Umbria,369,75,444,10941,11385,111,556,8889,293,5727.0,14840.0,20567,377286,210121.0


In [9]:
# New df regions state S - 1 with Trentino-Alto Adige
df_r_0 = Functions.replace_bt_with_taa(df_raw_r_0)
df_r_0.sort_values(by='codice_regione').reset_index()

,index,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati
0,0,2020-11-18T17:00:00,1,Piemonte,5208,388,5596,69468,75064,1278,3281,54511,5253,56287.0,78541.0,134828,1322809,836881.0
1,1,2020-11-18T17:00:00,2,Valle d'Aosta,146,17,163,1930,2093,-70,87,3234,259,4966.0,620.0,5586,51936,30337.0
2,2,2020-11-18T17:00:00,3,Lombardia,8323,903,9226,142857,152083,-4484,7633,169056,19850,257237.0,83752.0,340989,3644914,2193319.0
3,19,2020-11-18T17:00:00,4,Trentino-Alto Adige,879,77,956,12805,13761,66,817,17211,974,25667.0,6279.0,31946,620271,NaN
4,3,2020-11-18T17:00:00,5,Veneto,2079,274,2353,61953,64306,1235,2972,41613,3019,28335.0,80603.0,108938,2607348,1005997.0
5,4,2020-11-18T17:00:00,6,Friuli Venezia Giulia,470,49,519,10235,10754,333,796,9404,567,17665.0,3060.0,20725,636812,257516.0
6,5,2020-11-18T17:00:00,7,Liguria,1371,118,1489,15279,16768,237,775,26271,2163,31254.0,13948.0,45202,542028,271234.0
7,6,2020-11-18T17:00:00,8,Emilia-Romagna,2449,244,2693,56626,59319,2051,2371,31167,5168,61355.0,34299.0,95654,1916548,995472.0
8,7,2020-11-18T17:00:00,9,Toscana,1805,282,2087,52023,54110,-42,2508,30573,2022,68667.0,18038.0,86705,1385511,880609.0
9,8,2020-11-18T17:00:00,10,Umbria,368,74,442,10832,11274,30,501,8451,286,5646.0,14365.0,20011,371777,208149.0


In [10]:
df_r = df_r_1.copy()
# Merge with population data
df_pop_r = pd.read_csv(pop_regions_name)
df_m_r_1 = df_r.merge(df_pop_r, on='denominazione_regione')
df_m_r_1.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione
0,2020-11-19T17:00:00,1,Piemonte,5147,390,5537,71700,77237,2173,5349,57609,5331,58272.0,81905.0,140177,1364685,865428.0,4341375
1,2020-11-19T17:00:00,2,Valle d'Aosta,144,12,156,1879,2035,-58,91,3371,271,5042.0,635.0,5677,52636,30688.0,125501
2,2020-11-19T17:00:00,3,Lombardia,8291,915,9206,146374,155580,3497,7453,172847,20015,261506.0,86936.0,348442,3682509,2208742.0,10103969
3,2020-11-19T17:00:00,4,Trentino-Alto Adige,876,78,954,13236,14190,429,962,17724,994,26477.0,6431.0,32908,626406,NaN,1074819
4,2020-11-19T17:00:00,5,Veneto,2121,279,2400,64182,66582,2276,3753,43052,3057,28879.0,83812.0,112691,2624768,1011752.0,4907704
5,2020-11-19T17:00:00,6,Friuli Venezia Giulia,485,48,533,11112,11645,891,1197,9697,580,18558.0,3364.0,21922,643492,259446.0,1211357
6,2020-11-19T17:00:00,7,Liguria,1319,119,1438,15589,17027,259,792,26789,2178,31765.0,14229.0,45994,548215,273332.0,1543127
7,2020-11-19T17:00:00,8,Emilia-Romagna,2489,244,2733,58276,61009,1690,2160,31587,5218,62529.0,35285.0,97814,1935478,1003409.0,4467118
8,2020-11-19T17:00:00,9,Toscana,1814,287,2101,52023,54124,14,1972,32480,2073,69971.0,18706.0,88677,1403341,888853.0,3722729
9,2020-11-19T17:00:00,10,Umbria,369,75,444,10941,11385,111,556,8889,293,5727.0,14840.0,20567,377286,210121.0,880285


In [11]:
# Merge with icu data
df_icu_r = pd.read_csv(icu_regions_name)
df_m_r_2 = df_m_r_1.merge(df_icu_r, on='denominazione_regione')
df_m_r_2.sort_values(by='codice_regione').reset_index(drop=True)

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,dimessi_guariti,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,popolazione,totale_posti_terapia_intensiva
0,2020-11-19T17:00:00,1,Piemonte,5147,390,5537,71700,77237,2173,5349,57609,5331,58272.0,81905.0,140177,1364685,865428.0,4341375,575
1,2020-11-19T17:00:00,2,Valle d'Aosta,144,12,156,1879,2035,-58,91,3371,271,5042.0,635.0,5677,52636,30688.0,125501,20
2,2020-11-19T17:00:00,3,Lombardia,8291,915,9206,146374,155580,3497,7453,172847,20015,261506.0,86936.0,348442,3682509,2208742.0,10103969,1036
3,2020-11-19T17:00:00,4,Trentino-Alto Adige,876,78,954,13236,14190,429,962,17724,994,26477.0,6431.0,32908,626406,NaN,1074819,106
4,2020-11-19T17:00:00,5,Veneto,2121,279,2400,64182,66582,2276,3753,43052,3057,28879.0,83812.0,112691,2624768,1011752.0,4907704,1016
5,2020-11-19T17:00:00,6,Friuli Venezia Giulia,485,48,533,11112,11645,891,1197,9697,580,18558.0,3364.0,21922,643492,259446.0,1211357,180
6,2020-11-19T17:00:00,7,Liguria,1319,119,1438,15589,17027,259,792,26789,2178,31765.0,14229.0,45994,548215,273332.0,1543127,209
7,2020-11-19T17:00:00,8,Emilia-Romagna,2489,244,2733,58276,61009,1690,2160,31587,5218,62529.0,35285.0,97814,1935478,1003409.0,4467118,634
8,2020-11-19T17:00:00,9,Toscana,1814,287,2101,52023,54124,14,1972,32480,2073,69971.0,18706.0,88677,1403341,888853.0,3722729,523
9,2020-11-19T17:00:00,10,Umbria,369,75,444,10941,11385,111,556,8889,293,5727.0,14840.0,20567,377286,210121.0,880285,117


In [12]:
# Copy last merged df into df_r
df_r = df_m_r_2.copy()

In [13]:
# Add columns
df_r['variazione_ricoverati_con_sintomi'] = df_r_1['ricoverati_con_sintomi'] - df_r_0['ricoverati_con_sintomi']
df_r['variazione_terapia_intensiva'] = df_r_1['terapia_intensiva'] - df_r_0['terapia_intensiva']
df_r['variazione_totale_ospedalizzati'] = df_r_1['totale_ospedalizzati'] - df_r_0['totale_ospedalizzati']
df_r['posti_disponibili_terapia_intensiva'] = df_r['totale_posti_terapia_intensiva'] - df_r['terapia_intensiva']
df_r['saturazione_terapia_intensiva'] = ((df_r_1['terapia_intensiva'] / df_r['totale_posti_terapia_intensiva']) * 100).round(2)
df_r['variazione_isolamento_domiciliare'] = df_r_1['isolamento_domiciliare'] - df_r_0['isolamento_domiciliare']
df_r['variazione_nuovi_positivi'] = df_r_1['nuovi_positivi'] - df_r_0['nuovi_positivi']
df_r['nuovi_tamponi'] = df_r_1['tamponi'] - df_r_0['tamponi']
df_r['nuovi_casi_testati'] = df_r_1['casi_testati'] - df_r_0['casi_testati']
df_r['nuovi_deceduti'] = df_r_1['deceduti'] - df_r_0['deceduti']
df_r['nuovi_dimessi_guariti'] = df_r_1['dimessi_guariti'] - df_r_0['dimessi_guariti']
df_r['np_su_nt'] = ((df_r['nuovi_positivi'] / df_r['nuovi_tamponi']) * 100).round(2)
df_r['np_su_p'] = ((df_r['nuovi_positivi'] / df_r['popolazione']) * 100).round(5)
df_r['tc_su_p'] = ((df_r['totale_casi'] / df_r['popolazione']) * 100).round(2)

df_r = df_r.sort_values(by='nuovi_positivi', ascending=False).reset_index(drop=True)
df_r

,data,codice_regione,denominazione_regione,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,totale_positivi,variazione_totale_positivi,nuovi_positivi,...,saturazione_terapia_intensiva,variazione_isolamento_domiciliare,variazione_nuovi_positivi,nuovi_tamponi,nuovi_casi_testati,nuovi_deceduti,nuovi_dimessi_guariti,np_su_nt,np_su_p,tc_su_p
0,2020-11-19T17:00:00,3,Lombardia,8291,915,9206,146374,155580,3497,7453,...,88.32,3517,-180,37595,15423.0,165,3791,19.82,0.07376,3.45
1,2020-11-19T17:00:00,1,Piemonte,5147,390,5537,71700,77237,2173,5349,...,67.83,2232,2068,41876,28547.0,78,3098,12.77,0.12321,3.23
2,2020-11-19T17:00:00,5,Veneto,2121,279,2400,64182,66582,2276,3753,...,27.46,2229,781,17420,5755.0,38,1439,21.54,0.07647,2.30
3,2020-11-19T17:00:00,15,Campania,2287,194,2481,93705,96186,1047,3334,...,38.42,1025,-323,23496,11397.0,32,2255,14.19,0.05762,2.17
4,2020-11-19T17:00:00,12,Lazio,3103,329,3432,72109,75541,2050,2697,...,38.84,1992,-169,27519,23893.0,61,586,9.80,0.04598,1.62
5,2020-11-19T17:00:00,8,Emilia-Romagna,2489,244,2733,58276,61009,1690,2160,...,38.49,1650,-211,18930,7937.0,50,420,11.41,0.04835,2.19
6,2020-11-19T17:00:00,9,Toscana,1814,287,2101,52023,54124,14,1972,...,54.88,0,-536,17830,8244.0,51,1907,11.06,0.05297,2.38
7,2020-11-19T17:00:00,19,Sicilia,1532,240,1772,31809,33581,1479,1871,...,40.82,1475,34,11470,7475.0,40,352,16.31,0.03766,0.97
8,2020-11-19T17:00:00,16,Puglia,1324,200,1524,26976,28500,886,1263,...,54.20,880,-105,9386,7292.0,28,349,13.46,0.03151,0.98
9,2020-11-19T17:00:00,6,Friuli Venezia Giulia,485,48,533,11112,11645,891,1197,...,26.67,877,401,6680,1930.0,13,293,17.92,0.09881,1.81


In [14]:
# Write pre-processed data
df_r.to_csv(csv_out_name, index=False)

In [15]:
# ICU Stats
df_r[['denominazione_regione', 'terapia_intensiva', 'posti_disponibili_terapia_intensiva', 'totale_posti_terapia_intensiva', 'saturazione_terapia_intensiva']] \
    .sort_values(by='saturazione_terapia_intensiva', ascending=False).reset_index(drop=True)

,denominazione_regione,terapia_intensiva,posti_disponibili_terapia_intensiva,totale_posti_terapia_intensiva,saturazione_terapia_intensiva
0,Lombardia,915,121,1036,88.32
1,Trentino-Alto Adige,78,28,106,73.58
2,Piemonte,390,185,575,67.83
3,Umbria,75,42,117,64.10
4,Valle d'Aosta,12,8,20,60.00
5,Liguria,119,90,209,56.94
6,Toscana,287,236,523,54.88
7,Marche,78,65,143,54.55
8,Puglia,200,169,369,54.20
9,Abruzzo,74,74,148,50.00


In [16]:
def add_cols(df):
    df['variazione_terapia_intensiva'] = df['terapia_intensiva'].diff()
    df['variazione_ricoverati_con_sintomi'] = df['ricoverati_con_sintomi'].diff()
    df['variazione_totale_ospedalizzati'] = df['totale_ospedalizzati'].diff()
    df['nuovi_dimessi_guariti'] = df['dimessi_guariti'].diff()
    df['nuovi_deceduti'] = df['deceduti'].diff()
    df['nuovi_tamponi'] = df['tamponi'].diff()
    df['nuovi_casi_testati'] = df['casi_testati'].diff()
    df['variazione_isolamento_domiciliare'] = df['isolamento_domiciliare'].diff()
    df = df.iloc[1:]
    df['ti_su_to'] = ((df['terapia_intensiva'] / df['totale_ospedalizzati']).replace(np.inf, 0) * 100).round(2)
    df['to_su_tp'] = ((df['totale_ospedalizzati'] / df['totale_positivi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nt'] = ((df['nuovi_positivi'] / df['nuovi_tamponi']).replace(np.inf, 0) * 100).round(2)
    df['np_su_nct'] = ((df['nuovi_positivi'] / df['nuovi_casi_testati']).replace(np.inf, 0) * 100).round(2)
    df['var_ti_perc'] = ((df['variazione_terapia_intensiva'] / (df['terapia_intensiva'] - df['variazione_terapia_intensiva'])).replace(np.inf, 0) * 100).round(2)
    df['var_pos_perc'] = ((df['nuovi_positivi'] / (df['totale_positivi'] - df['nuovi_positivi'] + df['nuovi_dimessi_guariti'] + df['nuovi_deceduti'])).replace(np.inf, 0) * 100).round(3)
    return df.fillna(0)

In [17]:
# df_rd.sort_values(by='codice_regione').reset_index(drop=True)
df_rn = pd.read_csv(regions_name)
for region_code in df_rn.codice_regione:
    if region_code != 4: # TAA
        df_cur_region = df_rd[df_rd.codice_regione == region_code]
        region_name = df_cur_region.denominazione_regione.iloc[0].lower()
        csv_out_region_name = '{path_csv}/regioni/covid19-{region_name}.csv'.format(path_csv=path_csv, region_name=region_name)
        df_cur_region = add_cols(df_cur_region)
        df_cur_region.to_csv(csv_out_region_name, index=False)
        print('Aggiornato {file}'.format(file=csv_out_region_name))

Aggiornato ../data/csv/regioni/covid19-piemonte.csv
Aggiornato ../data/csv/regioni/covid19-valle d'aosta.csv
Aggiornato ../data/csv/regioni/covid19-lombardia.csv
Aggiornato ../data/csv/regioni/covid19-veneto.csv
Aggiornato ../data/csv/regioni/covid19-friuli venezia giulia.csv
Aggiornato ../data/csv/regioni/covid19-liguria.csv
Aggiornato ../data/csv/regioni/covid19-emilia-romagna.csv
Aggiornato ../data/csv/regioni/covid19-toscana.csv
Aggiornato ../data/csv/regioni/covid19-umbria.csv
Aggiornato ../data/csv/regioni/covid19-marche.csv
Aggiornato ../data/csv/regioni/covid19-lazio.csv
Aggiornato ../data/csv/regioni/covid19-abruzzo.csv
Aggiornato ../data/csv/regioni/covid19-molise.csv
Aggiornato ../data/csv/regioni/covid19-campania.csv
Aggiornato ../data/csv/regioni/covid19-puglia.csv
Aggiornato ../data/csv/regioni/covid19-basilicata.csv
Aggiornato ../data/csv/regioni/covid19-calabria.csv
Aggiornato ../data/csv/regioni/covid19-sicilia.csv
Aggiornato ../data/csv/regioni/covid19-sardegna.csv
